In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Install packages

In [ ]:
!pip -qq install langchain langchain_community langchain_pinecone

In [ ]:
!pip -qq install langchain_openai

In [ ]:
!pip -qq install python-docx pypdf PyPDF2

In [ ]:
!pip -qq install sentence_transformers

##import packages and pincone api key setting


In [ ]:
from langchain.chains import RetrievalQA
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_pinecone import PineconeEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain_community.document_loaders import PyPDFLoader
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from PyPDF2 import PdfReader
from docx import Document
import os

os.environ["PINECONE_API_KEY"] = "Enter Your API Key"

## sentence transformers to chunk the data by using model from huggingface

In [ ]:
def load_data_from_book(book_path):
    loader = PyPDFLoader(book_path)
    data = loader.load()
    return data

#create chunks of text
def split_text(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

#download embedding model
def download_embedding_model():
    embedding_model = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-MiniLM-L6-v2")
    return embedding_model

## Chating with LLM from pincone knowledge

In [ ]:
extracted_data = load_data_from_book("/content/drive/MyDrive/FinalYearProject/Medical_book.pdf")
text_chunks = split_text(extracted_data)
print(len(text_chunks))
embedding = download_embedding_model()
index_name = "medical-chatbot"

# Initialize a LangChain embedding object.
model_name = "gpt-3.5-turbo"
embeddings = PineconeEmbeddings(
    model=model_name,
    pinecone_api_key=os.environ.get("PINECONE_API_KEY")
)

# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embedding,
)

llm = ChatOpenAI(model_name="gpt-3.5-turbo",
openai_api_key="Enter Your API Key")

# Initialize a LangChain object for chatting with the LLM
# with knowledge from Pinecone.
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=docsearch.as_retriever()
)

5860


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
ERROR:asyncio:Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7b25c37e8c40>


## User Interface

In [ ]:
!pip -qq install pyngrok

In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -o ngrok-stable-linux-amd64.zip

--2024-11-06 03:27:01--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 99.83.220.108, 35.71.179.82, 13.248.244.96, ...
Connecting to bin.equinox.io (bin.equinox.io)|99.83.220.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13921656 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.1’

ngrok-stable-linux- 100%[===================>]  13.28M  7.76MB/s    in 1.7s    

2024-11-06 03:27:04 (7.76 MB/s) - ‘ngrok-stable-linux-amd64.zip.1’ saved [13921656/13921656]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [ ]:
!ngrok authtoken "Enter Your ngork authtoken"

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
#unzip the user interface files
!unzip -qq /content/drive/MyDrive/FinalYearProject/templates.zip


replace templates/chat_bot.html? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
# Configure the upload folder and allowed file extensions
UPLOAD_FOLDER = 'uploads/'
ALLOWED_EXTENSIONS = {'txt', 'pdf', 'docx'}

# Check if the uploaded file has an allowed extension
def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

# Function to read a text file
def read_txt(file):
    return file.read().decode('utf-8')

# Function to read a PDF file
def read_pdf(file):
    reader = PdfReader(file)
    text = ''
    for page in reader.pages:
        text += page.extract_text()
    return text

# Function to read a docx file
def read_docx(file):
    doc = Document(file)
    text = ''
    for para in doc.paragraphs:
        text += para.text + '\n'
    return text

# Replace with your Google Places API key
PLACES_API_KEY = "Enter Your API Key"

# Create the upload folder if it doesn't exist
if not os.path.exists(UPLOAD_FOLDER):
    os.makedirs(UPLOAD_FOLDER)

In [ ]:
from flask import Flask, render_template, request, flash, jsonify
import requests
from werkzeug.utils import secure_filename
from pyngrok import ngrok
import threading

# Initialize Flask app
app = Flask(__name__)
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

#home page
@app.route('/')
@app.route('/home')
def home():
    global health_tips
    # Static health tips data
    health_tips = [
        {"title": "Stay Hydrated", "text": "Drinking enough water is crucial for maintaining overall health. Aim for at least 8 glasses of water a day."},
        {"title": "Regular Exercise", "text": "Engage in at least 30 minutes of moderate exercise most days of the week to keep your body fit and healthy."},
        {"title": "Balanced Diet", "text": "Include a variety of fruits, vegetables, whole grains, and lean proteins in your diet to ensure you get all essential nutrients."}
    ]

    return render_template('home.html', health_tips=health_tips)

@app.route('/get_hospitals', methods=['POST'])
def get_hospitals():
    data = request.get_json()
    latitude = data.get('latitude')
    longitude = data.get('longitude')
    radius = 5000  # Search radius in meters

    # geoapify API request
    places_url = f"https://api.geoapify.com/v2/places?categories=healthcare&filter=circle:{longitude},{latitude},{radius}&limit=10&apiKey={PLACES_API_KEY}"
    response = requests.get(places_url)
    if response.status_code != 200:
        flash(f"Not able to find nearby hospitals due to {response.json()['message']}", 'warning')

    places_data = response.json()

    hospitals = []
    for place in places_data.get('features', []):
        details=place['properties']
        c_no='N/A'
        if "contact" in list(details.keys()):
            c_no=details["contact"]["phone"]
        hospitals.append({
            'Name': details.get('name'),
            'Address': details.get('formatted'),
            'Phone Number': c_no,
            'Website':details.get('website','N/A'),
            'Opening hours':details.get('opening_hours','N/A')
        })

    return jsonify({'hospitals': hospitals})


@app.route('/chatbot')
def chatbot():
    return render_template('chat_bot.html')

# Route to handle message input and optional file upload
@app.route('/send_message', methods=['POST'])
def send_message():
    message = request.form.get('message')  # Get the text input from the user
    file = request.files.get('file')  # Get the file if any

    file_content,combined_message = "",""

    # If a file is uploaded, process it based on its type
    if file:
        filename = file.filename
        if filename.endswith('.txt'):
            file_content = read_txt(file)
        elif filename.endswith('.pdf'):
            file_content = read_pdf(file)
        elif filename.endswith('.docx'):
            file_content = read_docx(file)
        else:
            return jsonify({'reply': 'Unsupported file type'})

    # Combine user message and file content (if any) before passing to LLM
    if len(file_content)!=0:
        combined_message = f"\n\nFile Content:\n'{file_content}' from this content answer this question '{message}'"

    response = {}  # Initialize a response dictionary

    # Generate a response for the message
    if len(combined_message)!=0 and qa:
      result = qa.invoke({"query":combined_message})
      response['reply'] = result["result"]
    elif qa:
      result=qa.invoke({"query":message})
      response['reply'] = result["result"]
    else:
      response['reply'] = "No response available"

    # Handle file upload if a file was uploaded
    if file and allowed_file(file.filename):
        filename = secure_filename(file.filename)
        file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
        response['file_info'] = f"File {filename} uploaded successfully."
    return jsonify(response)

# Function to run Flask app
def run_flask():
    app.run(port=5000)



# Expose the Colab environment via ngrok
public_url = ngrok.connect(5000)
print(f"Your Flask app is available at {public_url}")

# Start Flask app in the background
thread = threading.Thread(target=run_flask)
thread.start()

Your Flask app is available at NgrokTunnel: "https://2344-34-124-131-151.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
